In [ ]:
!git clone https://github.com/andreazenotto/tempProjectRepo.git

In [ ]:
%%capture
!apt update && apt install -y openslide-tools
!pip install openslide-python

In [ ]:
import gdown
import os
from tqdm import tqdm
import pandas as pd
import sys

sys.path.append('tempProjectRepo/mesothelioma_project/src')
from wsi_utils import load_wsi, extract_patches

## Show WSI-1 size

In [ ]:
import openslide

wsi_url = f"https://drive.google.com/uc?id=18HQP1e3IhTMaAyonylWxyaZiRaH5-M9e"
gdown.download(wsi_url)

# Carica la WSI
slide = openslide.OpenSlide("M-1.ndpi")

# Ottieni le dimensioni di tutti i livelli
levels = slide.level_dimensions

# Mostra le dimensioni per ogni livello
for i, (width, height) in enumerate(levels):
    print(f"Level {i} dimensions: {width} x {height}")

## Segmentation and Patching

In [ ]:
data_csv = "tempProjectRepo/mesothelioma_project/data/raw-data.csv"
output_dir = "tempProjectRepo/mesothelioma_project/data/patches"

diagnosis_map = {"E": "epithelioid", "S": "sarcomatoid", "B": "biphasic"}

df = pd.read_csv(data_csv, delimiter=r"\s+")

for _, row in tqdm(df.iterrows(), total=len(df)):
    filename = row['filename']
    gdrive_id = row['id']
    diagnosis_code = row['diagnosis']
    diagnosis_name = diagnosis_map[diagnosis_code]

    wsi_url = f"https://drive.google.com/uc?id={gdrive_id}"
    gdown.download(wsi_url, quiet=False)

    slide_id = os.path.splitext(filename)[0]
    slide_output_dir = os.path.join(output_dir, diagnosis_name, slide_id)

    # Load and process the WSI
    slide = load_wsi(filename)
    extract_patches(slide, slide_output_dir, level=0)

In [ ]:
from google.colab import files
import shutil

# Percorso della cartella da scaricare
folder_path = "tempProjectRepo/mesothelioma_project/data/patches"

# Nome del file zip da creare
zip_filename = "patches.zip"

# Comprimi la cartella
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_path)

# Scarica il file zip
files.download(zip_filename)